Parses data from leg info API and creates maps of legislators and session info

In [3]:
import requests
import xml.etree.ElementTree as ET

In [4]:
r = requests.Session()

endpoint = 'Legislators'
query = '?$select = SessionKey, LegislatorCode, FirstName, Party, DistrictNumber'
OLIS_API = 'https://api.oregonlegislature.gov/odata/odataservice.svc/'

legislators_response = r.get(OLIS_API+endpoint+query).text

In [5]:
content_tag = '{http://www.w3.org/2005/Atom}content'
properties_tag = '{http://schemas.microsoft.com/ado/2007/08/dataservices/metadata}properties'
data_tag = '{http://schemas.microsoft.com/ado/2007/08/dataservices}'

legislators = {}
sessions = {}

def get_session_info(session_key):
    '''
    Gets info about the session using session_key, returns tuple (begin date, end date, name)
    '''
    endpoint = 'LegislativeSessions(\'{}\')'.format(session_key)
    session = r.get(OLIS_API+endpoint).text
    root = ET.fromstring(session)
    name = ''
    for info in root.findall(".//{}*".format(data_tag)):
        if info.tag.endswith('SessionName'):
            name = info.text
        else:
            pass
    if len(name) > 0:
        sessions[session_key] = name 

def get_legislators_by_session(xml_tree):
    legs_by_session = {} # sessionkey -> sessionname -> legislators who served during that session
    
    for entry in xml_tree.findall(".//{http://www.w3.org/2005/Atom}entry"):
        name = []
        title = ''
        party = ''
        district = ''

        for properties in entry.findall("./{}/{}/{}*".format(content_tag, properties_tag, data_tag)):
            if properties.tag.endswith('SessionKey'): # p.text == session key
                session_key = properties.text
                if session_key not in sessions:
                    get_session_info(session_key)
            elif properties.tag.endswith('LegislatorCode'):
                leg_code = properties.text.split()
                if 'President' in leg_code or 'Speaker' in leg_code: # when the legislator is the sen. president
                    title = ' '.join(leg_code[:-1])
                    name.append(leg_code[-1])
                else:
                    title = leg_code[0]
                    name.extend(leg_code[1:])
            elif properties.tag.endswith('FirstName'):
                name.insert(0, properties.text)
            elif properties.tag.endswith('DistrictNumber'):
                # TODO map number to a city within
                district = properties.text
            else: # party affiliation
                party = properties.text
        
        leg_name = ' '.join(name)
        legislator = '{} {}, {}-{}'.format(title, leg_name, party[0], district)
        
        # add legislator to session[sessionkey][sessionname] = (start, end, set of legislators who were serving)
        sessionname = sessions[session_key]
        
        if session_key in legs_by_session:
            legs_by_session[session_key][sessionname].add(legislator)
        else:
            legs_by_session[session_key] = {
                sessionname: set({legislator}),
            }
        
        if leg_name in legislators:
            legislators[leg_name].add(session_key)
        else:
            legislators[leg_name] = {session_key}
                   
    return legs_by_session

In [6]:
root = ET.fromstring(legislators_response)
legs_by_session = get_legislators_by_session(root)

In [7]:
legislators

{'Teresa Alonso Leon': {'2017I1',
  '2017R1',
  '2018R1',
  '2018S1',
  '2019I1',
  '2019R1',
  '2020R1',
  '2020S1',
  '2020S2',
  '2020S3',
  '2021I1',
  '2021R1',
  '2021S1',
  '2021S2',
  '2022R1'},
 'Tom Andersen': {'2023I1',
  '2023R1',
  '2024R1',
  '2024S1',
  '2025I1',
  '2025R1',
  '2025S1'},
 'Jules Bailey': {'2007R1',
  '2009I1',
  '2009R1',
  '2010S1',
  '2011I1',
  '2011R1',
  '2012R1',
  '2012S1',
  '2013R1',
  '2013S1',
  '2014R1'},
 'Jeff Barker': {'2007I1',
  '2007R1',
  '2008S1',
  '2009I1',
  '2009R1',
  '2010S1',
  '2011I1',
  '2011R1',
  '2012R1',
  '2012S1',
  '2013I1',
  '2013R1',
  '2013S1',
  '2014R1',
  '2015I1',
  '2015R1',
  '2016R1',
  '2017I1',
  '2017R1',
  '2018R1',
  '2018S1',
  '2019I1',
  '2019R1',
  '2020R1',
  '2020S1',
  '2020S2',
  '2020S3'},
 'Phil Barnhart': {'2007I1',
  '2007R1',
  '2008S1',
  '2009I1',
  '2009R1',
  '2010S1',
  '2011I1',
  '2011R1',
  '2012R1',
  '2012S1',
  '2013I1',
  '2013R1',
  '2013S1',
  '2014R1',
  '2015I1',
  '2015R1'

In [8]:
sessions

{'2017I1': '2017-2018 Interim',
 '2017R1': '2017 Regular Session',
 '2018R1': '2018 Regular Session',
 '2018S1': '2018 1st Special Session',
 '2019I1': '2019-2020 Interim',
 '2019R1': '2019 Regular Session',
 '2020R1': '2020 Regular Session',
 '2020S1': '2020 1st Special Session',
 '2020S2': '2020 2nd Special Session',
 '2020S3': '2020 3rd Special Session',
 '2021I1': '2021-2022 Interim',
 '2021R1': '2021 Regular Session',
 '2021S1': '2021 1st Special Session',
 '2021S2': '2021 2nd Special Session',
 '2022R1': '2022 Regular Session',
 '2023I1': '2023-2024 Interim',
 '2023R1': '2023 Regular Session',
 '2024R1': '2024 Regular Session',
 '2024S1': '2024 1st Special Session',
 '2025I1': '2025-2026 Interim',
 '2025R1': '2025 Regular Session',
 '2025S1': '2025 1st Special Session',
 '2007R1': '2007 Regular Session',
 '2009I1': '2009 - 2010 Interim',
 '2009R1': '2009 Regular Session',
 '2010S1': '2010 Special Session',
 '2011I1': '2011 - 2012 Interim',
 '2011R1': '2011 Regular Session',
 '201

In [9]:
import json

def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    return ''

with open('./../data/legislators.json', 'w') as f:
    f.write(json.dumps(
    legislators,
    indent=4,
    separators=(',', ': '),
    default=set_default
    ))

with open('./../data/legs_by_session.json', 'w') as f:
    f.write(json.dumps(
    legs_by_session,
    indent=4,
    separators=(',', ': '),
    default=set_default
    ))